## 检测python版本
这里我们使用的python的版本为3.6.5

In [1]:
import sys
sys.version

'3.6.4 |Anaconda, Inc.| (default, Jan 16 2018, 18:10:19) \n[GCC 7.2.0]'

## 数据预处理
为了更好的训练模型，这里对数据进行一定程度对预处理
- 删除训练数据集中过大（`500*500`以上）和过小（`100*100`以下）的图片。
- 将图片按照一定的目录结构归类。
- 删除错误标记的图片。

In [2]:
import os, shutil
from PIL import Image

# 删除被错误标记的图片
def pick_err_flag(path, bad_path):

    # 没有目录，补充创建
    if not os.path.isdir(bad_path):
        os.mkdir(bad_path)
    
    bad_list = ['cat.92.jpg',    'cat.724.jpg',   'cat.1450.jpg',  'cat.3216.jpg', 'cat.3822.jpg', 'cat.5351.jpg',
                'cat.5418.jpg',  'cat.7377.jpg',  'cat.7564.jpg',  'cat.8456.jpg', 'cat.9171.jpg', 'cat.10029.jpg',
                'cat.10712.jpg', 'cat.11184.jpg', 'dog.1259.jpg',  'dog.1835.jpg', 'dog.2614.jpg', 'dog.3889.jpg',
                'dog.4367.jpg',  'dog.5604.jpg',  'dog.8736.jpg',  'dog.8898.jpg', 'dog.9517.jpg', 'dog.10161.jpg',
                'dog.10190.jpg', 'dog.10237.jpg', 'dog.10401.jpg', 'dog.10797.jpg','dog.10801.jpg','dog.11186.jpg',
                'dog.11299.jpg', 'dog.12376.jpg', 'dog.10747.jpg']
    for img_name in bad_list:
        im_path = os.path.join(path, img_name)
        if os.path.exists(im_path):
            shutil.move(im_path, os.path.join(bad_path, img_name))
    print(len(bad_list))

pick_err_flag("data/train", "data/train_bad")

33


In [3]:
# 删除不合尺寸
def pick_bad_pics(path, bad_path):

    # 没有目录，补充创建
    if not os.path.isdir(bad_path):
        os.mkdir(bad_path)
    
    bad_list = []
    img_list = os.listdir(path)
    for img_name in img_list:
        im_path = os.path.join(path, img_name)
        im = Image.open(im_path)
        w, h = im.size
        if w > 500 or h > 500 or w < 10 or h < 10:
            bad_list.append(img_name)
            shutil.move(im_path, os.path.join(bad_path, img_name))
    print(len(bad_list))

pick_bad_pics("data/train", "data/train_bad")

0


## 读取加载数据集，归一化处理
将用于训练的数据集加载到内存，等待处理。主要是转化为ndarray类型到数据，方便后续到计算和处理.因为选择到预训练模型，对于图片到要求都是`299*299`大小，这里我们读取数据时，图片统一调整到这个尺寸。
- 加载训练集数据。
- 加载测试集数据。
- 输出一个经过正规化的、Numpy array 格式的图像数据。

In [83]:
import glob, cv2
import numpy as np
from tqdm import tqdm

# 加载训练集
def load_train_data():

    cat = glob.glob("data/train/cat.*.jpg")
    dog = glob.glob("data/train/dog.*.jpg")
    train_data = np.zeros(((len(cat)+len(dog)), 299, 299, 3), dtype=np.uint8)
    train_targ = np.array([0]*len(cat) + [1]*len(dog))

    i = 0
    for img_name in tqdm(cat):
        img = cv2.imread(img_name)
        train_data[i] = cv2.resize(img,(299, 299))
        i += 1
    for img_name in tqdm(dog):
        img = cv2.imread(img_name)
        train_data[i] = cv2.resize(img,(299, 299))
        i += 1

    return train_data, train_targ

# 加载测试集
def load_test_data():

    test = glob.glob("data/test/*.jpg")
    test_data = np.zeros((len(test), 299, 299, 3), dtype=np.uint8)

    for img_name in tqdm(test):
        index = int(img_name[img_name.rfind('/')+1:img_name.rfind('.')])
        #print("index=%d name=%s" % (index, img_name))
        img = cv2.imread(img_name)
        test_data[index-1] = cv2.resize(img,(299, 299))
    
    return test_data

In [71]:
# 处理，加载训练集数据
train_data, train_targ = load_train_data()

100%|██████████| 12480/12480 [00:39<00:00, 313.53it/s]


In [84]:
# 处理，加载测试集数据
test_data = load_test_data()


100%|██████████| 12500/12500 [00:40<00:00, 311.92it/s]


In [85]:
print(train_data.shape, train_targ.shape, test_data.shape)

(24965, 299, 299, 3) (24965,) (12500, 299, 299, 3)


## 拆分验证集
对标记数据进行处理，拆分验证集

In [56]:
from sklearn.model_selection import train_test_split

# 划分数据
x_train, x_valid, y_train, y_valid = train_test_split(train_data, train_targ, test_size=0.2)

## 模型InceptionV3
预训练模型InceptionV3

In [9]:
import keras
import pandas as pd

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [10]:
# 构建InceptionV3
def buid_inceptionv3():

    # 获取基础模型，不保留顶层的全连接网络
    input_tensor = keras.Input(shape=(299, 299, 3)) 
    input_tensor = keras.layers.Lambda(keras.applications.inception_v3.preprocess_input)(input_tensor)
    base_model   = keras.applications.inception_v3.InceptionV3(input_tensor=input_tensor, include_top=False)

    # 锁定模型，保护处理
    for layer in base_model.layers:
        layer.trainable = False

    # 空域信号施加全局平均池化，dropout处理防止过拟合，重建全连接层
    x = keras.layers.GlobalAveragePooling2D()(base_model.output)
    x = keras.layers.Dropout(0.4)(x)
    x = keras.layers.Dense(1, activation='sigmoid')(x)

    # 配置模型
    result = keras.models.Model(inputs=base_model.input, outputs=x)
    result.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['accuracy'])

    # 返回
    print('InceptionV3 has %d layers.' % len(result.layers))
    return result

# 创建
inceptionv3_obj = buid_inceptionv3()

InceptionV3 has 315 layers.


In [11]:
# 可视化模型
#keras.utils.plot_model(inceptionv3_obj, to_file='model_inceptionv3.png')
#from IPython.display import SVG
#from keras.utils.vis_utils import model_to_dot
#SVG(model_to_dot(inceptionv3_obj).create(prog='dot', format='svg'))

In [12]:
# 训练
inceptionv3_obj.fit(x_train, y_train, batch_size=64, epochs=10, validation_data=(x_valid, y_valid))

Train on 19972 samples, validate on 4993 samples
Epoch 1/10
19972/19972 [==============================] - 291s 15ms/step - loss: 0.1987 - acc: 0.9325 - val_loss: 0.1156 - val_acc: 0.9579
Epoch 2/10
19972/19972 [==============================] - 285s 14ms/step - loss: 0.1103 - acc: 0.9620 - val_loss: 0.0936 - val_acc: 0.9674
Epoch 3/10
19972/19972 [==============================] - 285s 14ms/step - loss: 0.0934 - acc: 0.9650 - val_loss: 0.1784 - val_acc: 0.9345
Epoch 4/10
19972/19972 [==============================] - 285s 14ms/step - loss: 0.0855 - acc: 0.9676 - val_loss: 0.0593 - val_acc: 0.9828
Epoch 5/10
19972/19972 [==============================] - 285s 14ms/step - loss: 0.0857 - acc: 0.9674 - val_loss: 0.1073 - val_acc: 0.9650
Epoch 6/10
19972/19972 [==============================] - 285s 14ms/step - loss: 0.0819 - acc: 0.9692 - val_loss: 0.0836 - val_acc: 0.9740
Epoch 7/10
19972/19972 [==============================] - 285s 14ms/step - loss: 0.0852 - acc: 0.9675 - val_loss: 0.0

In [86]:
# 预测输出
inceptionv3_predict = inceptionv3_obj.predict(test_data)
inceptionv3_predict = inceptionv3_predict.clip(min=0.005, max=0.995)
inceptionv3_predict = inceptionv3_predict.flatten(order = 'F')

In [88]:
# 保存结果
submission = pd.DataFrame(data = {'id':(np.arange(len(test_data))+1), 'label': inceptionv3_predict})
submission.to_csv('submission_inceptionv3.csv',index=False)
submission.head(20)

,id,label
0,1,0.995000
1,2,0.995000
2,3,0.995000
3,4,0.989967
4,5,0.005000
5,6,0.005000
6,7,0.005000
7,8,0.005000
8,9,0.005000
9,10,0.005000


## 模型Xception
预训练模型Xception

In [14]:
# 构建xception
def buid_xception():

    # 获取基础模型，不保留顶层的全连接网络
    input_tensor = keras.Input(shape=(299, 299, 3)) 
    input_tensor = keras.layers.Lambda(keras.applications.xception.preprocess_input)(input_tensor)
    base_model   = keras.applications.xception.Xception(input_tensor=input_tensor, include_top=False)

    # 锁定模型，保护处理
    for layer in base_model.layers:
        layer.trainable = False

    # 空域信号施加全局平均池化，dropout处理防止过拟合，重建全连接层
    x = keras.layers.GlobalAveragePooling2D()(base_model.output)
    x = keras.layers.Dropout(0.4)(x)
    x = keras.layers.Dense(1, activation='sigmoid', kernel_initializer='he_normal')(x)

    # 配置模型
    result = keras.models.Model(inputs=base_model.input, outputs=x)
    result.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['accuracy'])

    # 返回
    print('xception has %d layers.' % len(result.layers))
    return result

# 创建模型
xception_obj = buid_xception()

xception has 136 layers.


In [15]:
# 可视化模型
#keras.utils.plot_model(xception_obj, to_file='model_xception.png')
#SVG(model_to_dot(xception_obj).create(prog='dot', format='svg'))

In [16]:
# 训练
xception_obj.fit(x_train, y_train, batch_size=64, epochs=10, validation_data=(x_valid, y_valid))

Train on 19972 samples, validate on 4993 samples
Epoch 1/10
19972/19972 [==============================] - 497s 25ms/step - loss: 0.1420 - acc: 0.9666 - val_loss: 0.1090 - val_acc: 0.9686
Epoch 2/10
19972/19972 [==============================] - 493s 25ms/step - loss: 0.0615 - acc: 0.9814 - val_loss: 0.0602 - val_acc: 0.9846
Epoch 3/10
19972/19972 [==============================] - 494s 25ms/step - loss: 0.0513 - acc: 0.9833 - val_loss: 0.0524 - val_acc: 0.9862
Epoch 4/10
19972/19972 [==============================] - 491s 25ms/step - loss: 0.0489 - acc: 0.9835 - val_loss: 0.0874 - val_acc: 0.9720
Epoch 5/10
19972/19972 [==============================] - 489s 25ms/step - loss: 0.0457 - acc: 0.9853 - val_loss: 0.0676 - val_acc: 0.9798
Epoch 6/10
19972/19972 [==============================] - 489s 25ms/step - loss: 0.0436 - acc: 0.9855 - val_loss: 0.0419 - val_acc: 0.9892
Epoch 7/10
19972/19972 [==============================] - 489s 24ms/step - loss: 0.0436 - acc: 0.9854 - val_loss: 0.0

In [90]:
# 预测输出
xception_predict = xception_obj.predict(test_data)
xception_predict = xception_predict.clip(min=0.005, max=0.995)
xception_predict = xception_predict.flatten(order = 'F')

In [91]:
# 保存结果
submission = pd.DataFrame(data = {'id':(np.arange(len(test_data))+1), 'label': xception_predict})
submission.to_csv('submission_xception.csv',index=False)
submission.head(20)

,id,label
0,1,0.987186
1,2,0.995000
2,3,0.985763
3,4,0.976971
4,5,0.005000
5,6,0.005000
6,7,0.005000
7,8,0.005000
8,9,0.005000
9,10,0.005000


## 模型Inception ResnetV2
预训练模型Inception ResnetV2

In [18]:
# 构建Inception ResnetV2
def buid_inception_resnet_v2():

    # 获取基础模型，不保留顶层的全连接网络
    input_tensor = keras.Input(shape=(299, 299, 3)) 
    input_tensor = keras.layers.Lambda(keras.applications.inception_resnet_v2.preprocess_input)(input_tensor)
    base_model = keras.applications.inception_resnet_v2.InceptionResNetV2(input_tensor=input_tensor, include_top=False)

    # 锁定模型，保护处理
    for layer in base_model.layers:
        layer.trainable = False

    # 空域信号施加全局平均池化，dropout处理防止过拟合，重建全连接层
    x = keras.layers.GlobalAveragePooling2D()(base_model.output)
    x = keras.layers.Dropout(0.4)(x)
    x = keras.layers.Dense(1, activation='sigmoid', kernel_initializer='he_normal')(x)

    # 配置模型
    result = keras.models.Model(inputs=base_model.input, outputs=x)
    result.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['accuracy'])

    # 返回
    print('Inception ResnetV2 has %d layers.' % len(result.layers))
    return result

# 创建
inception_resnet_v2_obj = buid_inception_resnet_v2()

Inception ResnetV2 has 784 layers.


In [19]:
# 可视化模型
#keras.utils.plot_model(inception_resnet_v2_obj, to_file='model_inception_resnet_v2.png')
#SVG(model_to_dot(inception_resnet_v2_obj).create(prog='dot', format='svg'))

In [20]:
# 训练
inception_resnet_v2_obj.fit(x_train, y_train, batch_size=64, epochs=10, validation_data=(x_valid, y_valid))

Train on 19972 samples, validate on 4993 samples
Epoch 1/10
19972/19972 [==============================] - 648s 32ms/step - loss: 0.1846 - acc: 0.9438 - val_loss: 0.0557 - val_acc: 0.9858
Epoch 2/10
19972/19972 [==============================] - 636s 32ms/step - loss: 0.0880 - acc: 0.9708 - val_loss: 0.0773 - val_acc: 0.9748
Epoch 3/10
19972/19972 [==============================] - 636s 32ms/step - loss: 0.0773 - acc: 0.9726 - val_loss: 0.0523 - val_acc: 0.9850
Epoch 4/10
19972/19972 [==============================] - 636s 32ms/step - loss: 0.0716 - acc: 0.9741 - val_loss: 0.0391 - val_acc: 0.9886
Epoch 5/10
19972/19972 [==============================] - 636s 32ms/step - loss: 0.0700 - acc: 0.9739 - val_loss: 0.1020 - val_acc: 0.9648
Epoch 6/10
19972/19972 [==============================] - 638s 32ms/step - loss: 0.0651 - acc: 0.9772 - val_loss: 0.0279 - val_acc: 0.9940
Epoch 7/10
19972/19972 [==============================] - 637s 32ms/step - loss: 0.0661 - acc: 0.9757 - val_loss: 0.0

In [92]:
# 预测输出
inception_resnet_v2_predict = inception_resnet_v2_obj.predict(test_data)
inception_resnet_v2_predict = inception_resnet_v2_predict.clip(min=0.005, max=0.995)
inception_resnet_v2_predict = inception_resnet_v2_predict.flatten(order = 'F')

In [93]:
# 保存结果
submission = pd.DataFrame(data = {'id':(np.arange(len(test_data))+1), 'label': inception_resnet_v2_predict})
submission.to_csv('submission_inception_resnet_v2.csv',index=False)
submission.head(20)

,id,label
0,1,0.995000
1,2,0.995000
2,3,0.995000
3,4,0.995000
4,5,0.005000
5,6,0.005000
6,7,0.005000
7,8,0.005000
8,9,0.005000
9,10,0.005000


## 提取特征，融合模型
将多个模型到特征向量融合训练

In [94]:
import h5py

# 提取模型的特征数据（原版本）
def pick_features(raw_model, pre_input):

    # 获取基础模型，不保留顶层的全连接网络
    inputs       = keras.Input(shape=(299, 299, 3))
    input_tensor = keras.layers.Lambda(pre_input)(inputs)
    base_model   = raw_model(input_tensor=input_tensor, include_top=False)

    # 提取特征数据
    x     = keras.layers.GlobalAveragePooling2D()(base_model.output)
    model = keras.models.Model(inputs=inputs, outputs=x)

    train_feature = model.predict(x_train, batch_size=64)
    test_feature  = model.predict(test_data, batch_size=64)

    # 返回
    with h5py.File("feature_%s.h5" % raw_model.__name__, 'w') as f:
        f.create_dataset('train', data=train_feature)
        f.create_dataset('test',  data=test_feature)
        f.create_dataset('label', data=y_train)
    #return train_feature, test_feature
    
    
# # 提取模型的特征数据（调整后）
# def pick_features(raw_model, pre_input, image_size=(299, 299)):

#     # 获取基础模型，不保留顶层的全连接网络
#     inputs       = keras.Input(shape=(image_size[0], image_size[1], 3))
#     input_tensor = inputs
#     if pre_input:
#         input_tensor = keras.layers.Lambda(pre_input)(inputs)
#     base_model   = raw_model(input_tensor=input_tensor, include_top=False)

#     # 提取特征数据
#     x     = keras.layers.GlobalAveragePooling2D()(base_model.output)
#     model = keras.models.Model(inputs=inputs, outputs=x)

#     gen = keras.preprocessing.image.ImageDataGenerator()
#     train_generator = gen.flow_from_directory("data/gen_train", image_size, shuffle=False, batch_size=16)
#     test_generator = gen.flow_from_directory("data/gen_test", image_size, shuffle=False, batch_size=16, class_mode=None)
#     train_feature = model.predict_generator(train_generator)
#     test_feature = model.predict_generator(test_generator) 
    
    
#     # 返回
#     with h5py.File("feature_%s.h5" % raw_model.__name__, 'w') as f:
#         f.create_dataset('train', data=train_feature)
#         f.create_dataset('test',  data=test_feature)
#         f.create_dataset('label', data=train_generator.classes)
#     #return train_feature, test_feature

In [ ]:
# 获取inceptionv3特征数据
pick_features(keras.applications.inception_v3.InceptionV3, keras.applications.inception_v3.preprocess_input)

In [ ]:
# 获取xception特征数据
pick_features(keras.applications.xception.Xception, keras.applications.xception.preprocess_input)

In [ ]:
# 获取inception_resnet_v2特征数据
pick_features(keras.applications.inception_resnet_v2.InceptionResNetV2, keras.applications.inception_resnet_v2.preprocess_input)

In [ ]:
# 获取ResNet50特征数据
#pick_features(keras.applications.resnet50.ResNet50, None, (244,244))

## 构建模型
构建最终的模型，用特征数据进行训练

In [ ]:
train, test= [], []
label = None
for fetchfile in ['feature_Xception.h5', 'feature_InceptionV3.h5', 'feature_InceptionResNetV2.h5']:
    with h5py.File(fetchfile, 'r') as h:
        train.append(np.array(h['train']))
        test.append(np.array(h['test']))
        label = np.array(h['label'])
        
train_data_feature = np.concatenate(train, axis=1)
test_data_feature  = np.concatenate(test, axis=1)

In [ ]:
from sklearn.utils import shuffle

# 随机处理
train_data_feature, label = shuffle(train_data_feature, label)

In [ ]:
x_ft_train, x_ft_valid, y_ft_train, y_ft_valid = train_test_split(train_data_feature, label, test_size=0.2)

In [ ]:
# 构建融合模型
def buid_final_model():

    inputs = keras.Input(shape=(x_ft_train.shape[1],))
    x = keras.layers.Dropout(0.4)(inputs)
    x = keras.layers.Dense(1, activation='sigmoid', kernel_initializer='he_normal')(x)

    # 配置模型
    final_model = keras.models.Model(inputs=inputs, outputs=x)
    final_model.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['accuracy'])

    # 返回
    return final_model


# 最终模型
final_model_obj = buid_final_model()

## 可视化展示
展示构建的最终模型结构

In [ ]:
# 可视化模型
#keras.utils.plot_model(final_model_obj, to_file='model.png')

#from IPython.display import SVG
#from keras.utils.vis_utils import model_to_dot
#SVG(model_to_dot(final_model_obj).create(prog='dot', format='svg'))

In [ ]:
import os

# 训练最终模型
def train_final_model(final_model):

    # 模型保存位置
    logs_file = 'ft_extract_features-{val_loss:.4f}.h5'
    path = os.getcwd()
    path_logs = os.path.join(path, logs_file)

    #early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
    #model_check = keras.callbacks.ModelCheckpoint(path_logs, monitor='val_loss', save_best_only=True)

    #final_model.fit(x_ft_train, y_ft_train, batch_size=64, epochs=10, 
    #           validation_data=(x_ft_valid, y_ft_valid), callbacks=[early_stop, model_check])
    final_model.fit(train_data_feature, label, batch_size=64, nb_epoch=10, validation_split=0.2)
    final_model.save(path_logs)
    

# 训练
train_final_model(final_model_obj)

In [ ]:
# 预测输出
final_predict = final_model_obj.predict(test_data_feature)
final_predict = final_predict.clip(min=0.005, max=0.995)
final_predict = final_predict.flatten(order = 'F')

#print(final_predict)
#print(test_data_feature)

# 保存结果
submission = pd.DataFrame(data = {'id':(np.arange(len(test_data))+1), 'label': final_predict})
submission.to_csv('submission_final.csv',index=False)
submission.head(20)

## 对数损失曲线
绘制模型的对数损失曲线，直观展示

In [ ]:
import matplotlib.pyplot as plt

# 对数损失更新曲线
def show_loss(final_model):
    fig, ax = plt.subplots(2,1)
    his_model = final_model.history
    history = his_model.history
    ax[0].plot(history['loss'], color='b', label="loss")
    ax[0].plot(history['val_loss'], color='r', label="val_loss",axes =ax[0])
    legend = ax[0].legend(loc='best', shadow=True)

    ax[1].plot(history['acc'], color='g', label="acc")
    ax[1].plot(history['val_acc'], color='c',label="val_acc")
    legend = ax[1].legend(loc='best', shadow=True)

# 绘制
show_loss(final_model_obj)